In [2]:
from munkres import Munkres
import copy
inf = 999999

In [3]:
#Если не удовлетворяет правилу треугольника, делаем чтобы удовлетворяло
def floyd_warshall_algorithm(matrix):
    n = len(matrix)
    matrix_new = copy.deepcopy(matrix)
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if (matrix_new[i][j] < inf):
                    matrix_new[i][j] = min(matrix_new[i][j], matrix_new[i][k]+matrix_new[k][j])
    return matrix_new

In [4]:
def Christofides_recurrent(input_matrix):
    matrix = floyd_warshall_algorithm(input_matrix)
    n = len(matrix)
    m = Munkres()            #Решаем задачу о назначениях
    idx = m.compute(matrix)
    way = dict(idx)
    #восстанавливаем циклы
    flags = [0]*n
    cycles = [[]]
    while sum(flags) < n:
        one_cycle = []
        i = flags.index(0)
        while flags[i] == 0:
            flags[i] = 1
            one_cycle.append(i)
            i = way[i]
        cycles.append(one_cycle)
    cycles = cycles[1:]
    if len(cycles) == 1:
        return cycles[0]
    #Объединяем циклы в точку и повторяем
    new_matrix = [[inf] * len(cycles) for i in range(len(cycles))]
    shortcuts = [[(-1, -1)] * len(cycles) for i in range(len(cycles))] 
    for i in range(len(cycles)):
        for j in range(len(cycles)):
            if(i != j):
                for p in range(len(cycles[i])):
                    for q in range(len(cycles[j])):
                        x = cycles[i][p]
                        y = cycles[j][q]
                        if(matrix[x][y] < new_matrix[i][j]):
                            new_matrix[i][j] = matrix[x][y]
                            shortcuts[i][j] = (x, y)
    external_cycle = Christofides_recurrent(new_matrix)
    #Объединяем циклы, выписывая порядки обхода малых циклов по порядку обхода большого
    external_cycle = external_cycle + external_cycle[0:2]
    perfect_cycle = []
    for i in range(1, len(external_cycle)-1):
        sequence_in = shortcuts[external_cycle[i-1]][external_cycle[i]][1]
        next_in = shortcuts[external_cycle[i]][external_cycle[i+1]][1]
        this_cycle = copy.deepcopy(cycles[external_cycle[i]])
        this_cycle = this_cycle + this_cycle[0:2]
        j = this_cycle[1:-1].index(sequence_in)+1
        direct_out = this_cycle[j-1]
        reverse_out = this_cycle[j+1]
        if(matrix[direct_out][next_in]-matrix[direct_out][sequence_in] <= matrix[reverse_out][next_in]-matrix[reverse_out][sequence_in]):
            perfect_cycle = perfect_cycle + this_cycle[j:-1]
            perfect_cycle = perfect_cycle + this_cycle[1:j]
        else:
            part_1 = this_cycle[1:j+1]
            part_1.reverse()
            part_2 = this_cycle[j+1:-1]
            part_2.reverse()
            perfect_cycle = perfect_cycle + part_1 + part_2
    return perfect_cycle

In [5]:
#тестим
#input_matrix = [[inf,10,15,19], [10, inf, 12, 17], [15, 12, inf, 8], [19, 17, 8, inf]]
input_matrix = [[inf,10, 15, 11, 2, 55], [10, inf, 16, 18, 21, 13], [15, 16, inf, 39, 22, 3], [11, 18,39, inf, 28, 25], [2, 21, 22, 28, inf, 2], [55, 13, 3, 25, 2, inf]]
input_matrix

[[999999, 10, 15, 11, 2, 55],
 [10, 999999, 16, 18, 21, 13],
 [15, 16, 999999, 39, 22, 3],
 [11, 18, 39, 999999, 28, 25],
 [2, 21, 22, 28, 999999, 2],
 [55, 13, 3, 25, 2, 999999]]

In [9]:
sequence = Christofides_recurrent(input_matrix)
sequence

[5, 2, 1, 3, 0, 4]

In [10]:
#2-opt
def algorithm_2opt(matrix, input_sequence):
    sequence = copy.deepcopy(input_sequence)
    flag_is_improved = 1
    while flag_is_improved == 1:
        flag_is_improved = 0
        for i in range(len(sequence)-2):
            for j in range(i+2,min(len(sequence)+i-1, len(sequence))):
                if (matrix[sequence[i]][sequence[i+1]] + matrix[sequence[j]][sequence[(j+1)%len(sequence)]] > matrix[sequence[i]][sequence[j]] + matrix[sequence[i+1]][sequence[(j+1)%len(sequence)]]):
                    rev_part = sequence[i+1:j+1]
                    rev_part.reverse()
                    sequence = sequence[:i+1] + rev_part + sequence[j+1:]
                    flag_is_improved = 1
    return sequence

In [11]:
sequence_2opt = algorithm_2opt(input_matrix, sequence)
sequence_2opt

[5, 2, 1, 3, 0, 4]

In [12]:
#запустимся начиная с рандомной последовательности и полуим на выходе ту же самую, с точностью до сдвига и разворота
sequence_2opt = algorithm_2opt(input_matrix, [0,1,2,3,4,5])
sequence_2opt

[0, 3, 1, 2, 5, 4]